# XGBoostのみで予測
## scoreはまだいい方だが、stackingと比べると下がる
## 特徴量エンジニアリングが効いている感じ

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8")) #check the files available in the directory

# データクレンジニング

In [3]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

train_id=train['Id']
test_id=test['Id']
# inplace=Trueにすれば変更を元のデータに反映させることが可能
train.drop("Id",axis=1,inplace=True)
test.drop("Id",axis=1,inplace=True)

train=train.drop(train[(train['GrLivArea']>4000) &(train['SalePrice']<300000)].index)
train['SalePrice']=np.log1p(train['SalePrice'])

y_train=train.SalePrice.values
print(y_train.shape)
#train['SalePrice'].values

all_data=pd.concat([train,test]).reset_index(drop=True)
all_data.drop(['SalePrice'],axis=1,inplace=True)

all_data_na=(all_data.isnull().sum()/len(all_data))*100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)

all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

# 中央値を欠損値に入れる groupbyでneghborhoodごとの中央値を取っている
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
    
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

all_data = all_data.drop(['Utilities'], axis=1)

train.groupby("Neighborhood").groups

print(all_data['Functional'].describe())
all_data["Functional"] = all_data["Functional"].fillna("Typ")

print(all_data['Electrical'].describe())
# mode()で最頻出を取得
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()


# .apply で関数を適用する
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)

# .astype でデータ型を変える
all_data['OverallCond'] = all_data['OverallCond'].astype(str)

all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

from sklearn.preprocessing import LabelEncoder
pd.options.display.max_columns = None
all_data.info()
# 特定のデータ型のみ取り出しselect_dtypes
all_data.select_dtypes(include=object)

cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for c in cols:
    lbl=LabelEncoder()
    lbl.fit(list(all_data[c].values))
    all_data[c]=lbl.transform(list(all_data[c].values))
print('Shape all_data: {}'.format(all_data.shape))

# 新しい特徴量を定義
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

# 数字の特徴量のカラムを取得
numeric_feats=all_data.dtypes[all_data.dtypes != 'object'].index

# 歪度を取得
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

skewness=skewness[abs(skewness)>0.75]
from scipy.special import boxcox1p

skewed_features=skewness.index
lam=0.15

for feat in skewed_features:
    all_data[feat]=boxcox1p(all_data[feat],lam)
    
all_data=pd.get_dummies(all_data)
print(all_data.shape)

ntrain=train.shape[0]
print(ntrain)
ntest=test.shape[0]
print(ntest)
train=all_data[:ntrain]
test=all_data[ntrain:]

X_train=train

(1458,)
count     2915
unique       7
top        Typ
freq      2715
Name: Functional, dtype: object
count      2916
unique        5
top       SBrkr
freq       2669
Name: Electrical, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2917 entries, 0 to 2916
Data columns (total 78 columns):
1stFlrSF         2917 non-null int64
2ndFlrSF         2917 non-null int64
3SsnPorch        2917 non-null int64
Alley            2917 non-null object
BedroomAbvGr     2917 non-null int64
BldgType         2917 non-null object
BsmtCond         2917 non-null object
BsmtExposure     2917 non-null object
BsmtFinSF1       2917 non-null float64
BsmtFinSF2       2917 non-null float64
BsmtFinType1     2917 non-null object
BsmtFinType2     2917 non-null object
BsmtFullBath     2917 non-null float64
BsmtHalfBath     2917 non-null float64
BsmtQual         2917 non-null object
BsmtUnfSF        2917 non-null float64
CentralAir       2917 non-null object
Condition1       2917 non-null object
Condition2  

In [18]:
import pickle
X_train=pd.read_pickle('X_train.pkl')
X_test=pd.read_pickle('X_test.pkl')
y_train=np.load('y_train.npy')
test=pd.read_csv('../input/test.csv')
test_id=test['Id']

In [23]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import optuna
xgb_reg=xgb.XGBRegressor(random_state=0)
parameters = {'learning_rate' : [0.1, 0.05, 0.02, 0.01], 
              'max_depth': [4, 6],
              'min_samples_leaf': [3, 5, 9, 17],
              'max_features': [1.0, 0.3, 0.1]}
model=GridSearchCV(estimator=xgb_reg,param_grid=parameters)
model.fit(X_train,y_train)

[21:34:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[21:34:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[21:35:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[21:35:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:35:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[21:36:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.05, 0.02, 0.01],
                         'max_depth': [4, 6], 'max

In [24]:
y_pred=np.expm1(model.predict(X_test))
sub=pd.DataFrame()
sub['Id']=test_id
sub['SalePrice']=y_pred
sub.to_csv('xgb_feature1_submission.csv')

In [27]:
sub

,Id,SalePrice
0,1461,117744.188
1,1462,163675.109
2,1463,191604.109
3,1464,193512.844
4,1465,195637.688
5,1466,168759.500
6,1467,172372.156
7,1468,161851.703
8,1469,189608.031
9,1470,122186.750


In [16]:
from sklearn.metrics import r2_score
# Objective Functionの作成
def opt(trial):
    n_estimators = trial.suggest_int('n_estimators', 0, 1000)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    subsample = trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1)
    colsample_bytree = trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1)
    xgboost_tuna = xgb.XGBRegressor(
        random_state=42,
        n_estimators = n_estimators,
        max_depth = max_depth,
        min_child_weight = min_child_weight,
        subsample = subsample,
        colsample_bytree = colsample_bytree,
    )
    xgboost_tuna.fit(X_train,y_train)
    tuna_pred_test = xgboost_tuna.predict(X_test)
    return (r2_score(y_test, tuna_pred_test))

In [17]:
import warnings
warnings.filterwarnings('ignore')
# 最適化
study = optuna.create_study()
study.optimize(opt, n_trials=100)

[21:26:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[W 2019-07-21 21:26:22,452] Setting status of trial#0 as TrialState.FAIL because of the following error: NameError("name 'y_test' is not defined",)
Traceback (most recent call last):
  File "/home/nagae/anaconda3/envs/gpu-env/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "<ipython-input-16-2f7f28fc19d2>", line 19, in opt
    return (r2_score(y_test, tuna_pred_test))
NameError: name 'y_test' is not defined


[21:26:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[W 2019-07-21 21:26:25,636] Setting status of trial#1 as TrialState.FAIL because of the following error: NameError("name 'y_test' is not defined",)
Traceback (most recent call last):
  File "/home/nagae/anaconda3/envs/gpu-env/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "<ipython-input-16-2f7f28fc19d2>", line 19, in opt
    return (r2_score(y_test, tuna_pred_test))
NameError: name 'y_test' is not defined


[21:26:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[W 2019-07-21 21:28:59,775] Setting status of trial#20 as TrialState.FAIL because of the following error: NameError("name 'y_test' is not defined",)
Traceback (most recent call last):
  File "/home/nagae/anaconda3/envs/gpu-env/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "<ipython-input-16-2f7f28fc19d2>", line 19, in opt
    return (r2_score(y_test, tuna_pred_test))
NameError: name 'y_test' is not defined


[21:29:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[W 2019-07-21 21:29:03,200] Setting status of trial#21 as TrialState.FAIL because of the following error: NameError("name 'y_test' is not defined",)
Traceback (most recent call last):
  File "/home/nagae/anaconda3/envs/gpu-env/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "<ipython-input-16-2f7f28fc19d2>", line 19, in opt
    return (r2_score(y_test, tuna_pred_test))
NameError: name 'y_test' is not defined


[21:29:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: 